# Name generation

In this challenge, you will generate names using recurrent neural networks!

The used dataset is in the file `names.txt`, a file encoded in `'ISO-8859-1'`, containing more than 10 000 names.

First load it, and have a look at the names, and clean the dataset if needed.

In [1]:
# TODO: Load the dataset and explore it
import pandas as pd
names=pd.read_csv('names.txt', encoding="ISO-8859-1")

names = names.drop_duplicates()

names.head()

,name
0,aaliyah
1,aapeli
2,aapo
3,aaren
4,aarne


The RNN needs to understand where is the beginning and the end of a word. So we need to add a new character at the beginning of every word, for example `'\t'` (it could be anything else as long as it can be identified easily). We can also add `'\n'` to the end of every word as the end.

In [2]:
# TODO: add '\t' at the beginning of every word
names['name'] =names["name"].apply(lambda x: '\t'+str(x)+'\n')

To generate names, we will have to play at the character level: we will train a RNN to predict the next character, knowing the previous one. So, compute a list of all the possible characters.

In [3]:
# TODO: Compute and display the list of all possible characters
# Get the vocab dict
all_chars=set()
for name in names.name:
    for c in name:
        if c not in all_chars:
            all_chars.add(c)
all_chars.add('\n')

print('number of characters', len(all_chars))
print(all_chars)

number of characters 55
{'ú', 'o', 'ö', 'ù', 't', 'å', 'è', 'n', 'ã', 'ê', 's', '\t', 'l', 'ð', 'b', 'à', 'f', 'p', 'r', 'a', 'g', '\n', 'ì', 'ä', 'æ', 'v', 'j', 'ï', 'ü', 'm', 'é', 'k', 'h', 'x', 'c', 'ø', 'e', 'ç', 'q', 'w', 'ñ', 'á', 'i', 'í', 'õ', 'u', 'd', 'þ', 'y', 'ò', 'z', 'ë', 'ô', '-', 'ó'}


You should get 55 characters, including the newly added `\t` and `\n`.

As usual when playing with characters (or words), we will convert them into integers. So build a dictionary `char_to_idx` that, given a character as key, returns an integer. And build the opposite dictionary `idx_to_char` that, given an integer as key, returns the corresponding character.

In [4]:
# TODO: Compute the idx_to_char and char_to_idx dict
char_to_idx = { ch:i for i,ch in enumerate(sorted(all_chars)) }
idx_to_char = { i:ch for i,ch in enumerate(sorted(all_chars)) }
char_to_idx

{'\t': 0,
 '\n': 1,
 '-': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 'à': 29,
 'á': 30,
 'ã': 31,
 'ä': 32,
 'å': 33,
 'æ': 34,
 'ç': 35,
 'è': 36,
 'é': 37,
 'ê': 38,
 'ë': 39,
 'ì': 40,
 'í': 41,
 'ï': 42,
 'ð': 43,
 'ñ': 44,
 'ò': 45,
 'ó': 46,
 'ô': 47,
 'õ': 48,
 'ö': 49,
 'ø': 50,
 'ù': 51,
 'ú': 52,
 'ü': 53,
 'þ': 54}

Before going into the neural network part, we have one more step: **create the X and y data**!

So the **X** data is going to be, for every name, all but the `'\n'` character. The **y** data will be all but the `'\t'` character.

Indeed, we will try to predict the following character knowing the previous. To the **X** does not need the final character, and the **y** does not need the first character.

Create the columns X and y to the dataframe.

In [5]:
# TODO: Create the columns X and y
names['X'] = names["name"].apply(lambda x: x[:len(x)-1])
names['y'] = names["name"].apply(lambda x: x[1:len(x)])
names.head()


,name,X,y
0,\taaliyah\n,\taaliyah,aaliyah\n
1,\taapeli\n,\taapeli,aapeli\n
2,\taapo\n,\taapo,aapo\n
3,\taaren\n,\taaren,aaren\n
4,\taarne\n,\taarne,aarne\n


Now, using your `char_to_idx` dict, compute the corresponding `X` and `y` containing, for each name, a list of integers.

In [6]:
# TODO: Create the X and y variables containing integers only
X = names['X'].apply(lambda x: [char_to_idx[c] for c in x])
y = names['y'].apply(lambda x: [char_to_idx[c] for c in x])

That was complicated, but are now in a known case, use keras and `pad_sequence()` function to get a proper `X` and `y` variables with a `maxlen=16` for example.

In [7]:
# TODO: Use pad_sequences to get only sequences of length 16 for each name
from tensorflow.keras.preprocessing import sequence

maxlen = 16

X_train = sequence.pad_sequences(X,
                                 value=0,
                                 padding='post',
                                 maxlen=maxlen)

y_train = sequence.pad_sequences(y,
                                 value=0,
                                 padding='post',
                                 maxlen=maxlen)
X_train.shape, y_train.shape

((11497, 16), (11497, 16))

Finally, using the function `to_categorical()`, make the one-hot-encoding needed.

In [8]:
# TODO: use to_categorical to perform one hot encoding
from tensorflow.keras.utils import to_categorical


X_train = to_categorical(X_train)
y_train = to_categorical(y_train)

X_train.shape, y_train.shape

((11497, 16, 55), (11497, 16, 55))

You should finally have arrays of shape `(number of names, 16, 55)`:
- `16` is the sequence length if you chose `maxlen=16`
- `55` is the number of possible characters

Now you have to build and train a neural network. You can for example use one or two layers of GRU (or LSTM). Do not forget to set `return_sequences=True`. 

Then you will have to add a `TimeDistributed(Dense(55))` with a softmax activation function. This layer will handle the fact you have a dense layer at each time step with a softmax prediction of the next word.

In [9]:
# TODO: Build the neural network and train it
from tensorflow.keras.layers import GRU, Dense, TimeDistributed, Embedding
from tensorflow.keras.models import Sequential

model = Sequential()
#model.add(Embedding(input_dim=55, input_length=16, output_dim=32))
model.add(GRU(32, input_shape=(maxlen, len(all_chars)), return_sequences=True))
model.add(GRU(32, return_sequences=True))
#model.add(GRU(32, return_sequences=True))
model.add(TimeDistributed(Dense(len(all_chars), activation='softmax')))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X_train, y_train, batch_size=64, epochs=50)

Train on 11497 samples
Epoch 1/50
11497/11497 [==============================] - 5s 476us/sample - loss: 2.0332
Epoch 2/50
11497/11497 [==============================] - 3s 246us/sample - loss: 1.4019
Epoch 3/50
11497/11497 [==============================] - 3s 263us/sample - loss: 1.2590
Epoch 4/50
11497/11497 [==============================] - 3s 272us/sample - loss: 1.1859
Epoch 5/50
11497/11497 [==============================] - 3s 277us/sample - loss: 1.1544
Epoch 6/50
11497/11497 [==============================] - 3s 241us/sample - loss: 1.1372
Epoch 7/50
11497/11497 [==============================] - 3s 273us/sample - loss: 1.1260
Epoch 8/50
11497/11497 [==============================] - 3s 272us/sample - loss: 1.1168
Epoch 9/50
11497/11497 [==============================] - 3s 283us/sample - loss: 1.1104
Epoch 10/50
11497/11497 [==============================] - 3s 274us/sample - loss: 1.1043
Epoch 11/50
11497/11497 [==============================] - 3s 242us/sample - loss: 1.0

The final step will be to generate names, through a function `generate_names()`. 

To do so, you will have to give the output of the previous time step prediction as input to the next time step.

You will have to use the method `predict_proba` of your model, as well as the method `numpy.random.choice`.

Finally, use your function to generate some names!

In [11]:
# TODO: implement the function generate_names
from generate import generate_n_names

generate_n_names(20, maxlen, char_to_idx, model)

	koriah
	arkun
	eardan
	betza
	frelino
	olfida
	òrbera
	yothayna
	mahredit
	grispalin
	ianne
	karja
	stilv
	aorisos
	malia
	evelami
	varluba
	sinnel
	haires
	rouslie


In case this looks too complicated (indeed it is not simple), you can use the function `generate_n_names()` in the file `generate.py`. But first have a look at it and try to understand what it does!

If you have more time, you can try to improve the results by tuning your neural network hyperparameters.

You can also use the original file, `Prenoms.csv`, and use only names from a given origin, to build a model more specific for example.

**Conclusion**: This method can be applied to almost anything: you can generate music, shakespeare, lyrics... All it takes is to change the data preprocessing and adapt the dimensions.